In [12]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

The % symbol denotes a magic command in Jupyter. It ensures that pip is run in the same Python environment as the Jupyter kernel. This prevents issues that might arise if the pip installed packages into a different environment.

In [ ]:
%pip install -U sentence-transformers
%pip install srsly
#%pip install triton - dont' need with mac m2
%pip install git+https://github.com/huggingface/transformers.git # install new huggingface transformers

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/vf/n84t7b8171lf64smq4ddvw1m0000gn/T/pip-req-build-o56afk73
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/vf/n84t7b8171lf64smq4ddvw1m0000gn/T/pip-req-build-o56afk73
  Resolved https://github.com/huggingface/transformers.git to commit 5fa35344755d8d9c29610b57d175efd03776ae9e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached huggingface_hub-0.27.1-py3-none-any.whl (450 kB)
Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl (2.6 MB)
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none

In [11]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/metal.html

import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: {device}")
else:
    print("MPS backend not available. Falling back to CPU.")

Using device: mps


In [14]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("joe32140/ModernBERT-base-msmarco")
# Note - for my medical project, seems like maybe using a different model would be better
# also may need to retinker the sentence transformer part

# Run inference
sentences = [
    'what county is hayden in',
    "Hayden is a city in Kootenai County, Idaho, United States. Located in the northern portion of the state, just north of Coeur d'Alene, its population was 13,294 at the 2010 census.",
    "According to the United States Census Bureau, the city has a total area of 9.61 square miles (24.89 km2), of which 9.60 square miles (24.86 km2) is land and 0.01 square miles (0.03 km2) is water. It lies at the southwestern end of Hayden Lake, and the elevation of the city is 2,287 feet (697 m) above sea level. Hayden is located on U.S. Route 95 at the junction of Route 41. It is also four miles (6 km) north of Interstate 90 and Coeur d'Alene. The Coeur d'Alene airport is northwest of Hayden.",
]

embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 768]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/273k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Compiling the model with `torch.compile` and using a `torch.mps` device is not supported. Falling back to non-compiled mode.


(3, 768)
torch.Size([3, 3])


In [15]:
#!pip install transformers[sentencepiece]
from transformers import pipeline

text = "Angela Merkel is a politician in Germany and leader of the CDU"
hypothesis_template = "This text is about {}" # Based on this interview, does the patient have {} sign or symptom
classes_verbalized = ["politics", "economy", "entertainment", "environment"]

zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Device set to use mps:0


In [16]:
%%time

zeroshot_classifier(text, classes_verbalized, hypothesis_template=hypothesis_template, multi_label=False)

CPU times: user 382 ms, sys: 901 ms, total: 1.28 s
Wall time: 4.17 s


{'sequence': 'Angela Merkel is a politician in Germany and leader of the CDU',
 'labels': ['politics', 'economy', 'environment', 'entertainment'],
 'scores': [0.9994651079177856,
  0.0002065976441372186,
  0.0001747352071106434,
  0.00015351509500760585]}

In [22]:
!wget https://raw.githubusercontent.com/koaning/arxiv-frontpage/refs/heads/main/data/annot/new-dataset.jsonl

--2025-01-18 22:11:19--  https://raw.githubusercontent.com/koaning/arxiv-frontpage/refs/heads/main/data/annot/new-dataset.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 402274 (393K) [text/plain]
Saving to: ‘new-dataset.jsonl’

new-dataset.jsonl   100%[===================>] 392.85K  --.-KB/s    in 0.09s   

2025-01-18 22:11:20 (4.28 MB/s) - ‘new-dataset.jsonl’ saved [402274/402274]



In [23]:
import srsly
inputs = list(srsly.read_jsonl("new-dataset.jsonl"))
texts = [d["text"] for d in inputs]
labels = [d["cats"]["new-dataset"] for d in inputs]

In [24]:
%%time

outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

CPU times: user 1min 52s, sys: 1min 25s, total: 3min 18s
Wall time: 11min 39s


In [26]:
import numpy as np
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

0.8990536277602523

In [27]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/ModernBERT-large-zeroshot-v2.0")

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/792M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Device set to use mps:0


In [28]:
%%time

zeroshot_classifier(text, classes_verbalized, hypothesis_template=hypothesis_template, multi_label=False)

CPU times: user 379 ms, sys: 1.35 s, total: 1.73 s
Wall time: 5.02 s


{'sequence': 'Angela Merkel is a politician in Germany and leader of the CDU',
 'labels': ['politics', 'economy', 'environment', 'entertainment'],
 'scores': [0.9994553327560425,
  0.0004048824484925717,
  0.0001086627526092343,
  3.1137329642660916e-05]}

In [29]:
%%time
outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

CPU times: user 1min 47s, sys: 1min 48s, total: 3min 35s
Wall time: 12min 17s


In [30]:
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

0.8940964398377648

In [31]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/ModernBERT-base-zeroshot-v2.0")

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/299M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Device set to use mps:0


In [32]:
%%time

outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

CPU times: user 1min 25s, sys: 7min 30s, total: 8min 55s
Wall time: 1h 4min 12s


In [ ]:
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)